# Data Cleaning and Preparation

McKinney, Wes. Python for data analysis: Data wrangling with Pandas, NumPy, and IPython. " O'Reilly Media, Inc.", 2012.

Chapter 7: Data Cleaning and Preparation 

Main source: https://github.com/wesm/pydata-book/blob/2nd-edition/ch07.ipynb

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
print(string_data)
string_data.isnull() #unique()

In [ ]:
string_data[0] = None
print(string_data)
string_data.isnull()

### Filtering Out Missing Data

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data.replace(NA, "four") #inplace=True

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
print(data,"\n")
print(cleaned)

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
print(data)
data.dropna(axis=1, how='all')

In [ ]:
np.random.seed(12345)
df = pd.DataFrame(np.random.rand(7, 3))
print(df,"\n")
df.iloc[:4, 1] = None
df.iloc[:2, 2] = NA
print(df)

In [ ]:
print(df.dropna(),"\n")
print(df.dropna(thresh=2)) #Keep only the rows with at least 2 non-NA values

### Filling In Missing Data

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0}) #you can treat each column differently

In [ ]:
df.fillna(0, inplace=True) # "inplace = True" means keep the DataFrame with valid entries in the same variable. 
df

In [ ]:
np.random.seed(12345)
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [ ]:
print(df.fillna(method='ffill'),"\n") #bfill

In [ ]:
print(df.fillna(method='ffill', limit=2)) # only apply the method twice on each column

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean()) # fill the missing value with mean of the data

In [ ]:
# You can define your own function to fill the Nan.
def fill_it(x):
    x.replace(NA, x.mean()-0.2, inplace=True)
    return (x)

In [ ]:
df.apply(lambda s: fill_it(s))

In [ ]:
#How to replace the missing values in the df with the mean of each column?
np.random.seed(12345)
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df


## Data Transformation

### Removing Duplicates

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated() #To identify duplicates

In [ ]:
data.drop_duplicates(inplace=False) #To drop duplicates

In [ ]:
data['v1'] = range(7)
print(data,"\n")
data.drop_duplicates(['k1']) #drop duplicates based on specific columns

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last') #In case it is important to keep the last row. 
                                                #The defualt is to keep the first row.

### Transforming Data Using a Function or Mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
lowercased = data['food'].str.lower() #convert everything to the lower type.
print(lowercased)

data['animal'] = lowercased.map(meat_to_animal) #mapping the food products to the relevant animal and store it in the animal column.
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) #all in one converting the food products to the lower type and mapping it to the relevant animal.

### Replacing Values

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0]) #list-liked

In [ ]:
data.replace({-999: np.nan, -1000: 0}) #dictionary-liked

### Renaming Axis Indexes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

In [ ]:
transform = lambda x: x[:4].upper() #Max four characters and upper case
data.index.map(transform)

In [ ]:
data.index = data.index.map(transform)
data

In [ ]:
data.rename(index=str.lower, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},     #renaming specific index and/or column
            columns={'three': 'peekaboo'}) 

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

In [ ]:
print(cats.codes) #codes: 0,1,2,3 since there are 4 categorize
print(cats.categories)
pd.value_counts(cats)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) #right side limit is not included

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
np.random.seed(12345)
data = np.random.rand(20)
print(data)
pd.cut(data, 4, precision=2)

In [ ]:
np.random.seed(12345)
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
pd.value_counts(cats)

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

### Detecting and Filtering Outliers

In [ ]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

In [ ]:
col = data[2]
col[np.abs(col) > 3]

In [ ]:
data[(np.abs(data) > 3).any(axis='columns')] #any(1)

In [ ]:
np.sign(data).head()

### Permutation and Random Sampling

In [ ]:
np.random.seed(12345)
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

In [ ]:
sampler = np.random.permutation(5)
sampler

In [ ]:
df.take(sampler)

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
print(choices)
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

print(df)
pd.get_dummies(df['key']) #To encode categorical features we are adding new features 
                          #corresponding to each unique element in the original feature. 
                          #One means the feature is active, zero means inactive.

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('./data/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

In [ ]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [ ]:
genres

In [ ]:
zero_matrix = np.zeros((len(movies), len(genres)))
print(zero_matrix)
dummies = pd.DataFrame(zero_matrix, columns=genres)
print(dummies)

In [ ]:
gen = movies.genres[0]
print(gen)
print(gen.split('|'))
dummies.columns.get_indexer(gen.split('|'))

In [ ]:
for i, gen in enumerate(movies.genres): # for each row of genres, i represents the index and gen is a string of genres separated by |
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [ ]:
dummies

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic#.iloc[0]

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
print(values)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## String Manipulation

### String Object Methods

In [ ]:
val = '  a,b  ,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')] #for each item x in the val split by ',' remove the extra whitespace
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
print(val)
print('guido' in val)
print(val.index(',')) #index of the word/character in the string, starting from 0
print(val.find(','))  #If the value is not found, the find() method returns -1, but 

In [ ]:
val.index(':') #If the value is not found, the index() method will raise an exception 

In [ ]:
val.index(',')

In [ ]:
val.count(',')

In [ ]:
print(val.replace(',', '::'))
val.replace(',', '')

### Regular Expressions

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text) #remove whitespaces one to many

In [ ]:
regex = re.compile('\s+') #remove whitespaces one to many
regex.split(text)

In [ ]:
print(regex.findall(text)) #find all types of whitespaces in the text, the pattern we are looking for

#regex = re.compile('[a-z]+')
#regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.se
Rob rob.a@gmail.com
Ryan ryan_a@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
print(m)
text[m.start():m.end()] #first location where the regular expression pattern produces

In [ ]:
print(regex.match(text)) #If zero or more characters at the beginning of string match the 
                         #regular expression pattern, return a corresponding match object. 
                         #Return None if the string does not match the pattern.
        

In [ ]:
# let's remove Dave from the beginning
text_1 = """dave@google.com 
Steve steve@gmail.se
Rob rob.a@gmail.com
Ryan ryan_a@yahoo.com
"""
print(regex.match(text_1))

In [ ]:
#we can use () for creating groups 
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) #assigning each group to its field

### Vectorized String Functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
print(data,"\n")
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
print(pattern)
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
print(data.str.get(1),"\n") #second character. Note that names are the indices.
print(data.str[0:])

In [ ]:
data.str[:5]

In [ ]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS